# Root level

In [11]:
%%writefile package.json
{
  "name": "nfl-kicker-assessment",
  "private": true,
  "scripts": {
    "start": "npm start --prefix backend",
    "dev": "concurrently \"npm start --prefix backend\" \"npm run dev --prefix frontend\"",
    "build": "npm run build --prefix frontend",
    "preview": "npm run build && npx serve -s frontend/dist",
    "ci:validate": "node scripts/ci-validate.js",
    "test": "npm test --prefix backend && npm test --prefix frontend"
  },
  "devDependencies": {
    "concurrently": "^8.0.0"
  }
} 

Overwriting package.json


In [12]:
%%writefile render.yaml
services:
  - type: web
    name: nfl-kicker-api
    env: node
    plan: free
    rootDir: backend
    buildCommand: "npm ci --omit=dev"
    startCommand: "node server.js"
    healthCheckPath: /api/ping    # Render watches this
    autoDeploy: true
    envVars:
      - key: CORS_ORIGIN
        value: "https://<your-site>.netlify.app"
      - key: NODE_ENV
        value: production 

Overwriting render.yaml


In [13]:
%%writefile netlify.toml
[build]
  command = "npm run build --prefix frontend"
  publish = "frontend/dist"

[[redirects]]
  from = "/*"
  to   = "/index.html"
  status = 200 

Overwriting netlify.toml


# github actions


In [14]:
%%writefile .github/workflows/netlify.yml
name: Netlify Deploy
on: { push: { branches: [ main ] } }

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with: { node-version: '18' }
      
      # Validate environment variables
      - name: Validate environment variables
        run: node scripts/ci-validate.js
        env:
          VITE_API_URL: ${{ secrets.VITE_API_URL }}
          CORS_ORIGIN: ${{ secrets.CORS_ORIGIN }}
      
      # Build and deploy
      - run: npm ci --prefix frontend
      - run: npm run build --prefix frontend
      - uses: netlify/actions/cli@master
        env:
          NETLIFY_AUTH_TOKEN: ${{ secrets.NETLIFY_TOKEN }}
          NETLIFY_SITE_ID: ${{ secrets.NETLIFY_SITE_ID }}
        with:
          args: deploy --dir=frontend/dist --prod 


Overwriting .github/workflows/netlify.yml


# Backend

In [15]:
%%writefile backend/package.json
{
  "name": "nfl-kicker-backend",
  "version": "1.0.0",
  "description": "Express backend for NFL Kicker Assessment",
  "main": "server.js",
  "type": "module",
  "scripts": {
    "start": "node server.js",
    "dev": "nodemon server.js"
  },
  "dependencies": {
    "express": "^4.18.2",
    "cors": "^2.8.5",
    "multer": "^1.4.5-lts.1",
    "csv-parser": "^3.0.0"
  },
  "devDependencies": {
    "nodemon": "^3.0.2"
  },
  "keywords": ["nfl", "kicker", "assessment", "api"],
  "author": "",
  "license": "MIT",
  "engines": {
    "node": ">=18.0.0"
  }
} 


Overwriting backend/package.json


In [16]:
%%writefile backend/server.js
import express from 'express';
import cors from 'cors';
import multer from 'multer';
import csv from 'csv-parser';
import fs from 'fs';
import path from 'path';
import { fileURLToPath } from 'url';

const __filename = fileURLToPath(import.meta.url);
const __dirname = path.dirname(__filename);

const app = express();

function getAllowedOrigins() {
  if (process.env.CORS_ORIGIN) {
    return process.env.CORS_ORIGIN.split(',').map(o => o.trim())
  }
  if (process.env.NODE_ENV === 'production') {
    return [/\.netlify\.app$/, /\.onrender\.com$/]   // regex works in cors ≥2.8
  }
  return ['http://localhost:5173']
}

// Middleware
app.use(cors({
  origin: getAllowedOrigins(),
  credentials: true,
}));

app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// In production, serve static files from the public directory
if (process.env.NODE_ENV === 'production') {
  const publicPath = path.join(__dirname, 'public');
  app.use(express.static(publicPath));
  
  // Serve index.html for any non-API routes (SPA fallback)
  app.get('*', (req, res, next) => {
    if (req.path.startsWith('/api/')) {
      return next();
    }
    res.sendFile(path.join(publicPath, 'index.html'));
  });
}

// Configure multer for file uploads
const storage = multer.diskStorage({
  destination: (req, file, cb) => {
    const uploadDir = 'uploads/';
    // Ensure uploads directory exists (Railway volume mount point)
    if (!fs.existsSync(uploadDir)) {
      fs.mkdirSync(uploadDir, { recursive: true });
    }
    cb(null, uploadDir);
  },
  filename: (req, file, cb) => {
    // Add timestamp to prevent filename conflicts
    const timestamp = Date.now();
    const originalName = file.originalname;
    cb(null, `${timestamp}-${originalName}`);
  }
});

const upload = multer({ 
  storage,
  limits: {
    fileSize: 10 * 1024 * 1024, // 10MB limit
    files: 1
  },
  fileFilter: (req, file, cb) => {
    // Only allow CSV files
    if (file.mimetype === 'text/csv' || file.originalname.endsWith('.csv')) {
      cb(null, true);
    } else {
      cb(new Error('Only CSV files are allowed'), false);
    }
  }
});

// API Routes

// Health check endpoint
app.get('/api/ping', (req, res) => {
  res.json({ 
    pong: true, 
    timestamp: new Date().toISOString(),
    environment: process.env.NODE_ENV || 'development'
  });
});

// CSV upload and processing endpoint
app.post('/api/upload', upload.single('file'), (req, res) => {
  if (!req.file) {
    return res.status(400).json({ error: 'No file uploaded' });
  }

  const results = [];
  const filePath = req.file.path;

  console.log(`Processing uploaded file: ${req.file.originalname}`);

  // Stream parse the CSV file
  fs.createReadStream(filePath)
    .pipe(csv())
    .on('data', (data) => {
      // Clean and validate data
      const cleanData = {};
      Object.keys(data).forEach(key => {
        const cleanKey = key.trim();
        const value = data[key]?.toString().trim();
        
        // Convert numeric fields
        if (['player_id', 'distance', 'made', 'week', 'year'].includes(cleanKey)) {
          cleanData[cleanKey] = isNaN(value) ? value : Number(value);
        } else {
          cleanData[cleanKey] = value;
        }
      });
      
      // Basic validation - ensure required fields are present
      if (cleanData.player_id && cleanData.player_name && 
          cleanData.distance !== undefined && cleanData.made !== undefined) {
        results.push(cleanData);
      }
    })
    .on('end', () => {
      // Clean up the uploaded file
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });

      console.log(`Processed ${results.length} records from ${req.file.originalname}`);
      
      res.json({ 
        success: true,
        data: results,
        rowCount: results.length,
        filename: req.file.originalname
      });
    })
    .on('error', (error) => {
      console.error('Error processing CSV:', error);
      
      // Clean up the uploaded file on error
      fs.unlink(filePath, (err) => {
        if (err) console.error('Error deleting uploaded file:', err);
      });
      
      res.status(500).json({ 
        error: 'Error processing CSV file',
        details: error.message
      });
    });
});

// Get processed leaderboard data (if we want to store it server-side)
app.get('/api/leaderboard', (req, res) => {
  // This could be enhanced to return cached/processed leaderboard data
  res.json({ 
    message: 'Leaderboard endpoint - not implemented yet',
    timestamp: new Date().toISOString()
  });
});

// Error handling middleware
app.use((error, req, res, next) => {
  console.error('Server error:', error);
  
  if (error instanceof multer.MulterError) {
    if (error.code === 'LIMIT_FILE_SIZE') {
      return res.status(400).json({ error: 'File too large (max 10MB)' });
    }
    if (error.code === 'LIMIT_UNEXPECTED_FILE') {
      return res.status(400).json({ error: 'Too many files uploaded' });
    }
  }
  
  res.status(500).json({ 
    error: 'Internal server error',
    details: process.env.NODE_ENV === 'development' ? error.message : undefined
  });
});

// 404 handler for API routes
app.use('/api/*', (req, res) => {
  res.status(404).json({ error: 'API endpoint not found' });
});

// Start server
const PORT = process.env.PORT || 5000;
const HOST = process.env.HOST || '0.0.0.0';

app.listen(PORT, HOST, () => {
  console.log(`🚀 NFL Kicker API server running on http://${HOST}:${PORT}`);
  console.log(`📁 Environment: ${process.env.NODE_ENV || 'development'}`);
  console.log(`🔧 CORS enabled for: ${process.env.CORS_ORIGIN || (process.env.NODE_ENV === 'production' ? 'production domains' : 'development (localhost:5173, localhost:3000)')}`);
  console.log(`📊 Health check: http://${HOST}:${PORT}/api/ping`);
  
  // Railway deployment info
  if (process.env.RAILWAY_ENVIRONMENT) {
    console.log(`🚂 Railway Environment: ${process.env.RAILWAY_ENVIRONMENT}`);
    console.log(`🌍 Railway Service: ${process.env.RAILWAY_SERVICE_NAME || 'Unknown'}`);
  }
});

// Graceful shutdown
process.on('SIGTERM', () => {
  console.log('SIGTERM received, shutting down gracefully');
  process.exit(0);
});

process.on('SIGINT', () => {
  console.log('SIGINT received, shutting down gracefully');
  process.exit(0);
}); 


Overwriting backend/server.js


#  Frontend

In [17]:
%%writefile frontend/package.json
{
  "name": "nfl-kicker-frontend",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "start": "serve -s dist -l $PORT",
    "lint": "eslint . --ext js,jsx --report-unused-disable-directives --max-warnings 0",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^18.2.0",
    "react-dom": "^18.2.0",
    "recharts": "^2.12.7",
    "lucide-react": "^0.436.0"
  },
  "devDependencies": {
    "@types/react": "^18.2.43",
    "@types/react-dom": "^18.2.17",
    "@vitejs/plugin-react": "^4.2.1",
    "autoprefixer": "^10.4.16",
    "eslint": "^8.55.0",
    "eslint-plugin-react": "^7.33.2",
    "eslint-plugin-react-hooks": "^4.6.0",
    "eslint-plugin-react-refresh": "^0.4.5",
    "postcss": "^8.4.32",
    "tailwindcss": "^3.3.6",
    "vite": "^5.0.8"
  },
  "engines": {
    "node": ">=18.0.0"
  }
} 


Overwriting frontend/package.json


In [18]:
%%writefile frontend/vite.config.js
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'

// https://vitejs.dev/config/
export default defineConfig(({ mode }) => {
  const env = loadEnv(mode, process.cwd(), '')
  const API = env.VITE_API_URL || 'http://localhost:5000'   // dev fallback

  return {
    plugins: [react()],
    define: {
      __API_URL__: JSON.stringify(API),                    // usable in code
    },
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/api': { target: API, changeOrigin: true, secure: false },
      },
    },
    build: { outDir: 'dist' },
  }
}) 


Overwriting frontend/vite.config.js


In [19]:
%%writefile frontend/src/App.jsx
import React, { useState, useEffect } from 'react';
import { Upload, Download, BarChart3, Users, Target, TrendingUp, FileText, Calculator } from 'lucide-react';
import { LineChart, Line, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ScatterChart, Scatter, ResponsiveContainer, BarChart, Bar } from 'recharts';
import ReactMarkdown from 'react-markdown';

const NFLKickerAssessment = () => {
  const [data, setData] = useState(null);
  const [processedData, setProcessedData] = useState(null);
  const [leaderboard, setLeaderboard] = useState([]);
  const [activeTab, setActiveTab] = useState('leaderboard');
  const [filterText, setFilterText] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);
  const [modelParams, setModelParams] = useState({
    minAttempts: 10,
    weightRecency: 0.3,
    weightDistance: 0.4,
    weightAccuracy: 0.3
  });

  const calculateKickerRating = (kickerData) => {
    if (!kickerData || kickerData.length === 0) return 0;
    
    // Calculate basic statistics
    const totalAttempts = kickerData.length;
    const totalMakes = kickerData.filter(kick => kick.made === 1).length;
    const accuracy = totalMakes / totalAttempts;
    
    // Distance-weighted accuracy
    const distanceWeightedScore = kickerData.reduce((sum, kick) => {
      const distanceWeight = Math.max(0, (kick.distance - 20) / 40); // Normalize distance
      return sum + (kick.made * (1 + distanceWeight));
    }, 0) / totalAttempts;
    
    // Recency weight (more recent kicks weighted higher)
    const maxWeek = Math.max(...kickerData.map(k => k.week));
    const recencyScore = kickerData.reduce((sum, kick) => {
      const recencyWeight = kick.week / maxWeek;
      return sum + (kick.made * recencyWeight);
    }, 0) / totalAttempts;
    
    // Composite rating
    const rating = (
      modelParams.weightAccuracy * accuracy +
      modelParams.weightDistance * distanceWeightedScore +
      modelParams.weightRecency * recencyScore
    ) * 100;
    
    return Math.round(rating * 100) / 100;
  };

  const processData = () => {
    if (!data) return;
    
    setLoading(true);
    try {
      // Group by player
      const playerGroups = {};
      data.forEach(row => {
        if (!playerGroups[row.player_id]) {
          playerGroups[row.player_id] = {
            player_id: row.player_id,
            player_name: row.player_name,
            attempts: []
          };
        }
        playerGroups[row.player_id].attempts.push(row);
      });
      
      // Calculate ratings and create leaderboard
      const leaderboardData = Object.values(playerGroups)
        .filter(player => player.attempts.length >= modelParams.minAttempts)
        .map(player => ({
          player_id: player.player_id,
          player_name: player.player_name,
          rating: calculateKickerRating(player.attempts),
          attempts: player.attempts.length,
          accuracy: player.attempts.filter(a => a.made === 1).length / player.attempts.length
        }))
        .sort((a, b) => b.rating - a.rating)
        .map((player, index) => ({ ...player, rank: index + 1 }));
      
      setLeaderboard(leaderboardData);
      setProcessedData(playerGroups);
    } catch (err) {
      setError('Error processing data: ' + err.message);
    } finally {
      setLoading(false);
    }
  };

  const handleFileUpload = async (event) => {
    const file = event.target.files[0];
    if (!file) return;

    if (file.type !== 'text/csv') {
      setError('Please upload a CSV file');
      return;
    }

    setLoading(true);
    setError(null);

    try {
      // Try backend API first
      const formData = new FormData();
      formData.append('file', file);
      
      const response = await fetch('/api/upload', {
        method: 'POST',
        body: formData,
      });

      if (response.ok) {
        const result = await response.json();
        setData(result.data);
      } else {
        throw new Error('Backend API not available');
      }
      
    } catch (err) {
      // Fallback to client-side processing
      console.log('Using client-side processing:', err.message);
      
      const reader = new FileReader();
      reader.onload = (e) => {
        try {
          const text = e.target.result;
          const rows = text.split('\n').filter(row => row.trim());
          const headers = rows[0].split(',').map(h => h.trim());
          const parsedData = rows.slice(1).map(row => {
            const values = row.split(',');
            const obj = {};
            headers.forEach((header, index) => {
              const value = values[index]?.trim();
              obj[header] = isNaN(value) ? value : Number(value);
            });
            return obj;
          });
          setData(parsedData);
          setError(null);
        } catch (parseErr) {
          setError('Error parsing CSV file: ' + parseErr.message);
        }
      };
      reader.readAsText(file);
    } finally {
      setLoading(false);
    }
  };

  const downloadLeaderboard = () => {
    const csv = [
      'player_id,player_name,rating,rank',
      ...leaderboard.map(row => `${row.player_id},${row.player_name},${row.rating},${row.rank}`)
    ].join('\n');
    
    const blob = new Blob([csv], { type: 'text/csv' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'leaderboard.csv';
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
  };

  const generateReport = () => {
    const report = `# NFL Kicker Rating Model - Technical Assessment

## Model Overview
This rating system evaluates NFL kickers based on:
- **Accuracy**: Base field goal percentage
- **Distance Performance**: Weighted by kick difficulty (distance)
- **Recency**: More recent performance weighted higher

## Model Parameters
- Minimum Attempts: ${modelParams.minAttempts}
- Accuracy Weight: ${modelParams.weightAccuracy}
- Distance Weight: ${modelParams.weightDistance}
- Recency Weight: ${modelParams.weightRecency}

## Top 10 Kickers (Week 6, 2018)
${leaderboard.slice(0, 10).map((k, i) => 
  `${i + 1}. ${k.player_name} - Rating: ${k.rating} (${k.attempts} attempts, ${(k.accuracy * 100).toFixed(1)}% accuracy)`
).join('\n')}

## Model Interpretation
- Rating Scale: 0-100 (higher is better)
- Considers both volume and efficiency
- Adjusts for kick difficulty and timing

## Potential Improvements
1. Incorporate situational factors (weather, pressure)
2. Add opponent strength adjustments
3. Include career trajectory analysis
4. Implement Bayesian updating for new kickers

## Next Steps
1. Validate model with out-of-sample data
2. Compare against Vegas odds or expert rankings
3. Implement confidence intervals
4. Add visualization for decision-makers
    `;
    
    const blob = new Blob([report], { type: 'text/markdown' });
    const url = window.URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'kicker_analysis_report.md';
    document.body.appendChild(a);
    a.click();
    window.URL.revokeObjectURL(url);
  };

  useEffect(() => {
    if (data) {
      processData();
    }
  }, [data, modelParams]);

  const TabButton = ({ id, label, icon: Icon, active }) => (
    <button
      onClick={() => setActiveTab(id)}
      className={`flex items-center px-4 py-2 rounded-lg transition-colors ${
        active 
          ? 'bg-orange-600 text-white' 
          : 'bg-gray-100 text-gray-700 hover:bg-gray-200'
      }`}
    >
      <Icon className="w-4 h-4 mr-2" />
      {label}
    </button>
  );

  // Filter leaderboard data
  const filteredLeaderboard = leaderboard.filter(k => 
    k.player_name.toLowerCase().includes(filterText.toLowerCase())
  );

  return (
    <div className="min-h-screen bg-gradient-to-br from-orange-50 to-blue-50 py-12">
      <div className="max-w-7xl mx-auto px-4">
        <h1 className="text-3xl font-bold text-gray-700 mb-6">NFL Kicker Assessment</h1>

        {/* Navigation */}
        <div className="flex flex-wrap gap-2 mb-6">
          <TabButton 
            id="leaderboard" 
            label="Leaderboard" 
            icon={BarChart3}
            active={activeTab === 'leaderboard'} 
            onClick={() => setActiveTab('leaderboard')} 
          />
          <TabButton 
            id="analysis" 
            label="Analysis" 
            icon={BarChart3}
            active={activeTab === 'analysis'} 
            onClick={() => setActiveTab('analysis')} 
          />
          <TabButton 
            id="paper" 
            label="Technical Paper" 
            icon={FileText}
            active={activeTab === 'paper'} 
            onClick={() => setActiveTab('paper')} 
          />
        </div>

        {/* Leaderboard Tab */}
        {activeTab === 'leaderboard' && (
          <div className="bg-white rounded-xl p-6 shadow-lg">
            <div className="flex justify-between items-center mb-4">
              <input 
                type="text"
                placeholder="Search player..."
                value={filterText}
                onChange={e => setFilterText(e.target.value)}
                className="border px-4 py-2 rounded-lg"
              />
              <button
                onClick={downloadLeaderboard}
                className="flex items-center gap-2 bg-orange-600 text-white px-4 py-2 rounded-lg hover:bg-orange-700"
              >
                <Download size={16} />
                Download CSV
              </button>
            </div>
            
            <div className="overflow-x-auto">
              <table className="w-full border-collapse">
                <thead>
                  <tr className="bg-gray-50">
                    <th className="text-left p-4 border-b-2 border-gray-200">Player Name</th>
                    <th className="text-left p-4 border-b-2 border-gray-200">Rating</th>
                    <th className="text-left p-4 border-b-2 border-gray-200">Success Rate</th>
                    <th className="text-left p-4 border-b-2 border-gray-200">Total Attempts</th>
                  </tr>
                </thead>
                <tbody>
                  {filteredLeaderboard.map((kicker, index) => (
                    <tr key={index} className="hover:bg-gray-50">
                      <td className="p-4 border-b border-gray-200">{kicker.player_name}</td>
                      <td className="p-4 border-b border-gray-200">{kicker.rating.toFixed(2)}</td>
                      <td className="p-4 border-b border-gray-200">{(kicker.accuracy * 100).toFixed(1)}%</td>
                      <td className="p-4 border-b border-gray-200">{kicker.attempts}</td>
                    </tr>
                  ))}
                </tbody>
              </table>
            </div>
          </div>
        )}

        {/* Analysis Tab */}
        {activeTab === 'analysis' && (
          <div className="grid md:grid-cols-2 gap-6">
            <div className="bg-white rounded-xl p-6 shadow-lg">
              <h3 className="text-xl font-semibold mb-4">Success Rate by Distance</h3>
              <BarChart width={500} height={300} data={processedData}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="distance" />
                <YAxis />
                <Tooltip />
                <Legend />
                <Bar dataKey="success_rate" fill="#ea580c" name="Success Rate" />
              </BarChart>
            </div>
            
            <div className="bg-white rounded-xl p-6 shadow-lg">
              <h3 className="text-xl font-semibold mb-4">Rating Trend</h3>
              <LineChart width={500} height={300} data={processedData}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="week" />
                <YAxis />
                <Tooltip />
                <Legend />
                <Line type="monotone" dataKey="rating" stroke="#2563eb" name="Rating" />
              </LineChart>
            </div>
          </div>
        )}

        {/* Technical Paper Tab */}
        {activeTab === 'paper' && (
          <div className="bg-white rounded-xl p-6 shadow-lg prose max-w-none">
            <ReactMarkdown>
              {generateReport()}
            </ReactMarkdown>
          </div>
        )}

      </div>
    </div>
  );
};

export default NFLKickerAssessment; 


Overwriting frontend/src/App.jsx


# Validation

In [20]:
%%writefile scripts/ci-validate.js
#!/usr/bin/env node

function validateEnvVars() {
  const apiUrl = process.env.VITE_API_URL;
  const corsOrigin = process.env.CORS_ORIGIN;

  if (!apiUrl) {
    console.error('❌ VITE_API_URL is not set');
    process.exit(1);
  }

  if (!corsOrigin) {
    console.error('❌ CORS_ORIGIN is not set');
    process.exit(1);
  }

  try {
    const apiDomain = new URL(apiUrl).hostname;
    const corsOrigins = corsOrigin.split(',').map(o => o.trim());
    
    // Check if the API domain is allowed by CORS_ORIGIN
    const isAllowed = corsOrigins.some(origin => {
      if (origin.startsWith('/') && origin.endsWith('/')) {
        // It's a regex pattern
        const regex = new RegExp(origin.slice(1, -1));
        return regex.test(apiDomain);
      }
      return new URL(origin).hostname === apiDomain;
    });

    if (!isAllowed) {
      console.error(`❌ API domain ${apiDomain} is not allowed by CORS_ORIGIN`);
      console.error(`   CORS_ORIGIN: ${corsOrigin}`);
      process.exit(1);
    }

    console.log('✅ Environment variables are consistent');
    process.exit(0);
  } catch (error) {
    console.error('❌ Invalid URL format in environment variables');
    console.error(error);
    process.exit(1);
  }
}

validateEnvVars(); 

Overwriting scripts/ci-validate.js
